In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/almond-types-classification/Almond.csv


In [2]:
df=pd.read_csv('/kaggle/input/almond-types-classification/Almond.csv')

In [3]:
df.head()

,Unnamed: 0,Length (major axis),Width (minor axis),Thickness (depth),Area,Perimeter,Roundness,Solidity,Compactness,Aspect Ratio,Eccentricity,Extent,Convex hull(convex area),Type
0,0,NaN,227.940628,127.759132,22619.0,643.813269,NaN,0.973384,1.458265,NaN,NaN,0.681193,23237.5,MAMRA
1,1,NaN,234.188126,128.199509,23038.0,680.984841,NaN,0.957304,1.601844,NaN,NaN,0.656353,24065.5,MAMRA
2,2,NaN,229.418610,125.796547,22386.5,646.943212,NaN,0.967270,1.487772,NaN,NaN,0.683620,23144.0,MAMRA
3,3,NaN,232.763153,125.918808,22578.5,661.227483,NaN,0.965512,1.540979,NaN,NaN,0.685360,23385.0,MAMRA
4,4,NaN,230.150742,107.253448,19068.0,624.842706,NaN,0.951450,1.629395,NaN,NaN,0.714800,20041.0,MAMRA


In [4]:
print(df.isnull().sum())
print(df.duplicated().sum())

Unnamed: 0                     0
Length (major axis)          857
Width (minor axis)           942
Thickness (depth)           1004
Area                           0
Perimeter                      0
Roundness                    857
Solidity                       0
Compactness                    0
Aspect Ratio                1799
Eccentricity                1799
Extent                         0
Convex hull(convex area)       0
Type                           0
dtype: int64
0


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier 
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [9]:
df=df.drop(columns=['Unnamed: 0'])

In [10]:
numeric_columns=df.select_dtypes(include='number').columns

In [13]:
df[numeric_columns]=df[numeric_columns].fillna(df[numeric_columns].mean())
df['Type']=LabelEncoder().fit_transform(df['Type'])

In [14]:
X=df.drop(columns=['Type'])
y=df['Type']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
scaler=StandardScaler()

In [16]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [19]:
models={
    'DecisionTreeClassifier':DecisionTreeClassifier(random_state=42),
    'RandomForestClassifier':RandomForestClassifier(n_estimators=100,random_state=42),
    'GradientBoostingClassifier':GradientBoostingClassifier(n_estimators=100,random_state=42),
    'AdaBoostClassifier':AdaBoostClassifier(n_estimators=100,random_state=42),
    'XGBClassifier':XGBClassifier(n_estimators=100,random_state=42),
    'CatBoostClassifier':CatBoostClassifier(n_estimators=100,random_state=42,verbose=False),
    'LGBMClassifier': LGBMClassifier(n_estimators=100, random_state=42, verbose=0)
}

In [22]:
for name,model in models.items():
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    accuracy=accuracy_score(y_test,y_pred)
    report=classification_report(y_test,y_pred)
    conf_matrix=confusion_matrix(y_test,y_pred)
    print(f"{name}: Accuracy = {accuracy}")
    print(f"{name}: Classification Report:\n {report}")
    print(f"{name}: Confusion Matrix:\n {conf_matrix}")
    print('-----------------------------------------------')

DecisionTreeClassifier: Accuracy = 0.7522281639928698
DecisionTreeClassifier: Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.80      0.82       189
           1       0.73      0.73      0.73       187
           2       0.70      0.72      0.71       185

    accuracy                           0.75       561
   macro avg       0.75      0.75      0.75       561
weighted avg       0.75      0.75      0.75       561

DecisionTreeClassifier: Confusion Matrix:
 [[152  15  22]
 [ 15 137  35]
 [ 16  36 133]]
-----------------------------------------------
RandomForestClassifier: Accuracy = 0.8449197860962567
RandomForestClassifier: Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.90      0.91       189
           1       0.80      0.80      0.80       187
           2       0.82      0.84      0.83       185

    accuracy                           0.84       561
   mac